In [49]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel

In [50]:
spark = SparkSession.builder.master("local").getOrCreate()

In [51]:
movie_ratings = spark.read.json('./data/ratings.json')
movie_ratings = movie_ratings.toPandas()

In [52]:
movie_ratings.head()

,movie_id,rating,timestamp,user_id
0,858,4,956678732.0,6040
1,2384,4,956678754.0,6040
2,593,5,956678754.0,6040
3,1961,4,956678777.0,6040
4,1419,3,956678856.0,6040


In [53]:
import pandas as pd

In [54]:
movies_meta = pd.read_csv('features.csv')

In [55]:
movies_meta

,id,title_y,category,budget,revenue,original_title,runtime,vote_average,status,popularity,vote_count
0,1.0,Toy Story,"['Animation', ""Children's"", 'Comedy']",30000000,373554033,Toy Story,81.0,7.7,Released,21.946943,5415
1,2.0,Jumanji,"['Adventure', ""Children's"", 'Fantasy']",65000000,262797249,Jumanji,104.0,6.9,Released,17.015539,2413
2,3.0,Grumpier Old Men,"['Comedy', 'Romance']",0,0,Grumpier Old Men,101.0,6.5,Released,11.7129,92
3,4.0,Waiting to Exhale,"['Comedy', 'Drama']",16000000,[{'name': 'Twentieth Century Fox Film Corporat...,Waiting to Exhale,"[{'iso_3166_1': 'US', 'name': 'United States o...",Friends are the people who let you be yourself...,81452156,Bernie,Waiting to Exhale
4,5.0,Father of the Bride Part II,['Comedy'],0,76578911,Father of the Bride Part II,106.0,5.7,Released,8.387519,173
5,6.0,Heat,"['Action', 'Crime', 'Thriller']",60000000,187436818,Heat,170.0,7.7,Released,17.924927,1886
6,7.0,Sabrina,"['Comedy', 'Romance']",58000000,0,Sabrina,127.0,6.2,Released,6.677277,141
7,915.0,Sabrina,"['Comedy', 'Romance']",58000000,0,Sabrina,127.0,6.2,Released,6.677277,141
8,8.0,Tom and Huck,"['Adventure', ""Children's""]",0,0,Tom and Huck,97.0,5.4,Released,2.561161,45
9,9.0,Sudden Death,['Action'],35000000,64350171,Sudden Death,106.0,5.5,Released,5.23158,174


In [57]:
users_meta = pd.read_csv('users.csv')

In [58]:
users_meta.head()

,ID,sex,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [63]:
movies = pd.merge(movie_ratings, movies_meta, left_on='movie_id', right_on='id')

In [64]:
movies_and_users = pd.merge(movies, users_meta, left_on='user_id', right_on='ID')

In [65]:
movies_and_users.drop(['id','ID'], axis=1, inplace=True)

In [66]:
movies_and_users.head()

,movie_id,rating,timestamp,user_id,title_y,category,budget,revenue,original_title,runtime,vote_average,status,popularity,vote_count,sex,age,occupation,zipcode
0,2384,4,956678754.0,6040,Babe: Pig in the City,"[""Children's"", 'Comedy']",90000000,69131860,Babe: Pig in the City,92.0,5.3,Released,6.229629,312,M,25,6,11106
1,1961,4,956678777.0,6040,Rain Man,['Drama'],25000000,412800000,Rain Man,133.0,7.6,Released,11.267467,1762,M,25,6,11106
2,1419,3,956678856.0,6040,Walkabout,['Drama'],0,0,Walkabout,95.0,7.4,Released,10.177086,111,M,25,6,11106
3,3111,5,956678856.0,6040,Places in the Heart,['Drama'],0,0,Places in the Heart,111.0,7.1,Released,3.712567,31,M,25,6,11106
4,3505,4,956678856.0,6040,No Way Out,['Thriller'],15000000,35509515,No Way Out,114.0,6.9,Released,11.457803,155,M,25,6,11106
